### Сам себе Шпилькин

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('commission.xlsx')

In [3]:
data.head()

,№,ФИО,Статус,Кем предложен в состав комиссии,№ УИК
0,1,Пелина Ирина Николаевна,Председатель,собрание избирателей по месту работы,2001
1,2,Изуграфова Светлана Геннадиевна,Зам.председателя,собрание избирателей по месту работы,2001
2,3,Толочко Ирина Анатольевна,Секретарь,Региональное отделение в Краснодарском крае Вс...,2001
3,4,Билань Роман Васильевич,Член,Краснодарское местное отделение Краснодарского...,2001
4,5,Боглаевская Ольга Станиславовна,Член,собрание избирателей по месту работы,2001


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 5 columns):
№                                  128 non-null int64
ФИО                                128 non-null object
Статус                             128 non-null object
Кем предложен в состав комиссии    128 non-null object
№ УИК                              128 non-null int64
dtypes: int64(2), object(3)
memory usage: 5.1+ KB


In [5]:
data['№ УИК'].unique()

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008])

##### Информация о комиссии представлена с 8 избирательных участков

In [6]:
data['Кем предложен в состав комиссии'].unique()

array(['собрание избирателей по месту работы',
       'Региональное отделение в Краснодарском крае Всероссийской политической партии "Партия пенсионеров России"',
       'Краснодарское местное отделение Краснодарского регионального отделения Всероссийской политической партии "ЕДИНАЯ РОССИЯ"',
       'Краснодарское краевое отделение политической партии "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ"',
       'Региональное отделение Политической партии СПРАВЕДЛИВАЯ РОССИЯ в Краснодарском крае',
       'Краснодарское региональное отделение Политической партии ЛДПР - Либерально-демократической партии России',
       'Региональное отделение ВСЕРОССИЙСКОЙ ПОЛИТИЧЕСКОЙ ПАРТИИ "РОДИНА" в Краснодарском крае',
       'Региональное отделение Всероссийской политической партии "Гражданская Сила" в Краснодарском крае',
       'Общероссийская общественная организация "Российский Союз ветеранов Афганистана"',
       'Региональное отделение Политической партии "Казачья партия Российской Федерации" в Кра

In [7]:
def kprf(row):
    if 'КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ' in row:
        return 1
    else:
        return 0       

###### Проверяем нет ли различных названий партии КПРФ и вводим функцию, которая определят количество представителей КПРФ на каждом избирательном участке

In [8]:
data['КПРФ'] = data['Кем предложен в состав комиссии'].apply(kprf)

In [9]:
data_pivot = data.pivot_table(index = '№ УИК', values = 'КПРФ', aggfunc = 'sum')
data_pivot

,КПРФ
№ УИК,
2001,1
2002,1
2003,1
2004,0
2005,1
2006,1
2007,0
2008,1


In [10]:
kprf = []
for i in data_pivot.loc[data_pivot['КПРФ'] != 0].index:
    kprf.append(i)

In [11]:
not_kprf = []
for i in data_pivot.loc[data_pivot['КПРФ'] == 0].index:
    not_kprf.append(i)

In [12]:
print(*not_kprf, sep = ', ')

2004, 2007


##### Выводим номера избирательных участков, в которых нет представителя от КПРФ

In [13]:
report = pd.read_excel('report.xls')
report.head()

,Unnamed: 0,Unnamed: 1,Сумма,УИК №2001,УИК №2002,УИК №2003,УИК №2004,УИК №2005,УИК №2006,УИК №2007,УИК №2008
0,1,"Число избирателей, внесенных в список",18257,2517,2164,969,2592,2316,2670,2439,2583
1,2,"Число бюллетеней, полученных участковой комиссией",15607,2100,1800,900,2100,2100,2300,2100,2200
2,3,"Число бюллетеней, выданных избирателям, прогол...",4089,469,813,582,451,683,451,279,361
3,4,"Число бюллетеней, выданных участковой комиссие...",2111,292,394,17,389,239,331,257,185
4,5,"Число бюллетеней, выданных избирателям, прогол...",617,23,0,0,30,8,14,167,375


##### Информация о результатах  по каждому избирательному участку

In [14]:
report = report.loc[21:]
report

,Unnamed: 0,Unnamed: 1,Сумма,УИК №2001,УИК №2002,УИК №2003,УИК №2004,УИК №2005,УИК №2006,УИК №2007,УИК №2008
21,17,"Политическая партия ""КОММУНИСТИЧЕСКАЯ ПАРТИЯ Р...",901,89,119,72,139,98,165,61,158
22,NaN,NaN,13.39%,11.35%,9.90%,12.02%,16.35%,10.54%,20.75%,8.68%,18.37%


In [15]:
report_kprf = []
for i in kprf:
    report_kprf.append(float(report['УИК №'+str(i)].loc[22].replace('%', '')))
result_kprf = pd.Series(report_kprf).mean().round(2)
result_kprf

13.82

In [16]:
report_not_kprf = []
for i in not_kprf:
    report_not_kprf.append(float(report['УИК №'+str(i)].loc[22].replace('%', '')))
result_not_kprf = pd.Series(report_not_kprf).mean().round(2)
result_not_kprf

12.52

##### Вычисляем среднее значения результатов партии КПРФ на участках, где в избирательных комиссиях были представители от КПРФ и где таких представителей не было

In [17]:
print(result_kprf, result_not_kprf, abs((result_kprf-result_not_kprf).round(2)), sep=', ')

13.82, 12.52, 1.3


##### Таким образом, результат партии КПРФ на участках, где в избирательных комиссиях был представитель КПРФ оказался на 1,3% выше, чем без представителя